In [1]:
import azureml.core
from azureml.core import Workspace

ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.20.0 to work with azure-ml-ws


In [2]:
from azureml.core import Experiment
from azureml.core import Model
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# Create an Azure ML experiment
experiment = Experiment(workspace = ws, name = "echonest-song-classification")
run = experiment.start_logging()
print("Starting experiment:", experiment.name)

# Loading the dataset
print("Loading Data...")
echonest = pd.read_csv('datasets/echonest_data.csv')

X, y = echonest[['acousticness','danceability','energy','instrumentalness','liveness','speechiness','tempo','valence']].values, echonest['labels'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a decision tree model
print('Training a decision tree model')
model = DecisionTreeClassifier().fit(X_train, y_train)

Starting experiment: echonest-song-classification
Loading Data...
Training a decision tree model


In [3]:
# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# Save the trained model
model_file = 'echonest_song_model.pkl'
joblib.dump(value=model, filename=model_file)
run.upload_file(name = 'outputs/' + model_file, path_or_stream = './' + model_file)

# Complete the run
run.complete()

# Register the model
run.register_model(model_path='outputs/echonest_song_model.pkl', model_name='echonest_song_model',
                   tags={'Training context':'Inline Training'},
                   properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

print('Model trained and registered.')

Accuracy: 0.8764746703678001
AUC: 0.8062912170957255
Model trained and registered.


In [4]:
from azureml.core import Model

# Get Model properties
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

echonest_song_model version: 6
	 Training context : Inline Training
	 AUC : 0.8062912170957255
	 Accuracy : 0.8764746703678001


echonest_song_model version: 5
	 Training context : Inline Training
	 AUC : 0.810736031684341
	 Accuracy : 0.8792505204718946


echonest_song_model version: 4
	 Training context : Inline Training
	 AUC : 0.7938173585278132
	 Accuracy : 0.8695350451075642


echonest_song_model version: 3
	 Training context : Inline Training
	 AUC : 0.802848876303987
	 Accuracy : 0.8709229701596114


echonest_song_model version: 2
	 Training context : Inline Training
	 AUC : 0.7527406091339244
	 Accuracy : 0.84941013185288


echonest_song_model version: 1
	 Training context : Inline Training
	 AUC : 0.7477945576468702
	 Accuracy : 0.8480222068008327




In [5]:
import os

model = ws.models['echonest_song_model']
print(model.name, 'version', model.version)

folder_name = 'echonest-song-service'

# Create a folder for the web service files
experiment_folder = './' + folder_name
os.makedirs(experiment_folder, exist_ok=True)

print(folder_name, 'folder created.')

# Set path for scoring script
script_file = os.path.join(experiment_folder,"score_echonest_song_model.py")

echonest_song_model version 6
echonest-song-service folder created.


In [6]:
%%writefile $script_file
import json
import joblib
import numpy as np
from azureml.core.model import Model

# Called when the service is loaded
def init():
    global model
    # Get the path to the deployed model file and load it
    model_path = Model.get_model_path('echonest_song_model')
    model = joblib.load(model_path)

# Called when a request is received
def run(raw_data):
    # Get the input data as a numpy array
    data = np.array(json.loads(raw_data)['data'])
    # Get a prediction from the model
    predictions = model.predict(data)
    # Return the predictions as JSON
    return json.dumps(predictions.tolist())


Overwriting ./echonest-song-service/score_echonest_song_model.py


In [7]:
from azureml.core.conda_dependencies import CondaDependencies 

# Create a yaml file for all the scoring script dependencies. (Need scikit-learn for this case)
# Add the dependencies for our model (AzureML defaults is already included)
myenv = CondaDependencies()
myenv.add_conda_package('scikit-learn')

# Save the environment config as a .yml file
env_file = os.path.join(experiment_folder,"echonest_env.yml")
with open(env_file,"w") as f:
    f.write(myenv.serialize_to_string())
print("Saved dependency info in", env_file)

# Print the .yml file
with open(env_file,"r") as f:
    print(f.read())

Saved dependency info in ./echonest-song-service/echonest_env.yml
# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
    # Required packages for AzureML execution, history, and data preparation.
  - azureml-defaults

- scikit-learn
channels:
- anaconda
- conda-forge



In [8]:
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig

# Create an Azure Container Instance for the running the webservice.
# Configure the scoring environment
inference_config = InferenceConfig(runtime= "python",
                                   entry_script=script_file,
                                   conda_file=env_file)

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

service_name = "echonest-song-service"

service = Model.deploy(ws, service_name, [model], inference_config, deployment_config)

service.wait_for_deployment(True)
print(service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running.................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [19]:
import json

x_new = X_test[:5].tolist()

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({'data':x_new})

In [20]:
# Call the Azure ML web service, passing the input data
predictions = service.run(input_json)
predicted_classes = json.loads(predictions)

# Get the predictions
for i in range(len(x_new)):
    print (f"Song Genre: {predicted_classes[i]}")

Song Genre: Rock
Song Genre: Rock
Song Genre: Rock
Song Genre: Rock
Song Genre: Hip-Hop


In [21]:
endpoint = service.scoring_uri
print(endpoint)

http://d1429a94-a5c4-4e29-afa5-2c193545e951.eastus2.azurecontainer.io/score


In [22]:
import requests

# Call web service with simple HTTP request. 
x_new = X_test[:5].tolist()

input_json = json.dumps({"data": x_new})

# Set the content type
headers = { 'Content-Type':'application/json' }

predictions = requests.post(endpoint, input_json, headers = headers)
predicted_classes = json.loads(predictions.json())

for i in range(len(x_new)):
    print (f'Song Genre: {predicted_classes[i]}')

Song Genre: Rock
Song Genre: Rock
Song Genre: Rock
Song Genre: Rock
Song Genre: Hip-Hop
